# ДЗ 7

На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

# РЕШЕНИЕ

In [1]:
from scipy.stats import ttest_1samp
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import shapiro
from scipy import stats
from statsmodels.stats.power import TTestIndPower

Matplotlib is building the font cache; this may take a moment.


In [20]:
df = pd.read_excel('C:\Users\Ванечка\Desktop\MyReps\PythonData\ABTesting\008\AB_Test_Results.xlsx', encoding='utf-8')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3632356535.py, line 1)

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.VARIANT_NAME.value_counts()

Избавимся от некорретных строк. Посчитаем, сколько типов встречается у каждого пользователя.

In [ ]:
v = df.\
    groupby('USER_ID', as_index=False).\
    agg({'VARIANT_NAME': pd.Series.nunique})

In [ ]:
v.head(10)

In [ ]:
new_df = df.drop_duplicates(subset='USER_ID')
new_df

In [ ]:
alpha = 0.05

st = shapiro(new_df.REVENUE)
print('Distribution is {}normal\n'.format( {True:'not ',
False:''}[st[1] < alpha]))

In [ ]:
control = new_df.query('VARIANT_NAME == "control"')

test = new_df.query('VARIANT_NAME == "variant"')

In [ ]:
control.REVENUE.hist(bins = 25, alpha =0.7, label='Control')
test.REVENUE.hist(bins = 25, alpha =0.7, label='Test')
plt.title('Доход на юзера по группам')
plt.xlabel('Доход')
plt.ylabel('Число пользователей')
plt.legend();

In [ ]:
plt.boxplot([control.REVENUE,test.REVENUE],
            labels=['Контроль','Тест'],
            widths=0.5
           )
plt.title('Boxplot по пользователям',  loc='center')
plt.grid(axis  ='both')

In [ ]:
n_control = new_df[new_df['VARIANT_NAME']=='control'].shape[0]
n_control

In [ ]:
n_test = new_df[new_df['VARIANT_NAME']=='variant'].shape[0]
n_test

In [ ]:
stats.ttest_ind(control.REVENUE.values, test.REVENUE.values, equal_var = False)

In [ ]:
C_mean = control.REVENUE.values.mean()

T_mean = test.REVENUE.values.mean()


C_std = control.REVENUE.values.std()

T_std = test.REVENUE.values.std()

In [ ]:
S = np.sqrt((T_std**2 + C_std **2)/ 2)

In [ ]:
effect_size =float((T_mean-C_mean)/ S)

Определим мощность теста

In [ ]:
analysis = TTestIndPower()
result = analysis.solve_power(effect_size=effect_size, power=None, nobs1=n_test, ratio=1.0, alpha=alpha)

result

# ВЫВОД: 
pvalue > alpha. 

Изменений в двух группах не наблюдается. 

Также мощность теста варьируется в пределах 13%, что является низким значением. 

# РЕКОМЕНДАЦИИ:

1. Тест следует откатить назад.
2. Нужно собрать больше данных,
3. Увеличить длительность теста, а затем можно запустить заново.